<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-07-30 14:18:03


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)
my_prospects = list(df_mypf[(df_mypf['LatestQtr'] == 1) & df_mypf['StarStock'] == 1]['Symbol'].values)

In [6]:
def print_distribution():

  # star stocks
  print('Overall - Star Stocks:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('Overall - V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Overall - Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Overall - Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

  # stocks in portfolio
  print('Portfolio - Stocks:')
  print(df_mypf['InPortfolio'].value_counts())
  print('-------')

  # star stocks in portfolio
  print('Portfolio - Star Stocks:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())
  print('-------')

  # latest quarter in portfolio
  print('Portfolio - Latest Quarter:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['LatestQtr'].value_counts())
  print('-------')

  # criteria of portfolio stocks
  print('Portfolio - Criteria:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())
  print('-------')

  # strategies in portfolio stocks
  print('Portfolio - Strategy:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())
  print('-------')

  # conviction distribution
  print('Portfolio - Conviction:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Overall - Star Stocks:
1.0    96
0.0    89
Name: StarStock, dtype: int64
-------
Overall - V40/V40N Stocks:
V40N    39
V40     38
VR      23
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
-------
Overall - Latest Qtr:
0    111
1     76
Name: LatestQtr, dtype: int64
-------
Overall - Star + V40/V40N Stocks:
1.0    43
0.0    34
Name: StarStock, dtype: int64
-------
Portfolio - Stocks:
0    135
1     52
Name: InPortfolio, dtype: int64
-------
Portfolio - Star Stocks:
1.0    33
0.0    19
Name: StarStock, dtype: int64
-------
Portfolio - Latest Quarter:
0    31
1    21
Name: LatestQtr, dtype: int64
-------
Portfolio - Criteria:
VR      23
V40N    11
V40     11
AR       4
V200     3
Name: Criteria, dtype: int64
-------
Portfolio - Strategy:
ATH     36
2T3Y    14
BTT      2
Name: Strategy, dtype: int64
-------
Portfolio - Conviction:
M    27
L    14
H    11
Name: Conviction, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-06-30 00:00:00+05:30,19189.0,17954.0,6.88,^NSEI,19979.0,72.0
2023-07-07 00:00:00+05:30,19332.0,17990.0,7.46,^NSEI,19979.0,68.0
2023-07-14 00:00:00+05:30,19564.0,18035.0,8.48,^NSEI,19979.0,73.0
2023-07-21 00:00:00+05:30,19745.0,18105.0,9.06,^NSEI,19979.0,67.0
2023-07-28 00:00:00+05:30,19646.0,18167.0,8.14,^NSEI,19979.0,60.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

#df_common_features['Stock'] = df_common_features['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_common_features['Discount%PE'] = round((df_common_features['Med_PE']-df_common_features['Curr_PE'])*100/(df_common_features['Med_PE']),0)
df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
df_common_features['Upside%Target'] = round((df_common_features['Target']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features = df_common_features.set_index('Stock')

In [13]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Upside%Target',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'RSI_14', 'Curr_ROCE%', 'Curr_ROE%', 'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_common_features[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 52


,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1292.0,-7.71,202.0,1225.87,45.2,25.90,74.0,14.80,10.90,L,43.0,0,0.0,VR,2T3Y
VALIANTORG.NS,517.0,-4.81,200.0,532.61,24.0,16.50,48.0,14.80,13.00,L,31.0,0,0.0,VR,2T3Y
BANDHANBNK.NS,217.0,-7.62,176.0,244.52,NaN,17.20,39.0,6.67,11.90,M,NaN,1,0.0,VR,2T3Y
NIACL.NS,122.0,10.55,173.0,117.07,NaN,19.10,58.0,4.86,4.06,L,NaN,0,0.0,VR,ATH
LUXIND.NS,1555.0,3.38,170.0,1493.72,21.4,34.00,59.0,12.70,9.95,M,-59.0,0,0.0,V40N,2T3Y
VAIBHAVGBL.NS,339.0,7.29,158.0,317.48,35.4,53.30,65.0,11.10,9.02,L,-51.0,0,0.0,VR,2T3Y
PNB.NS,62.0,23.90,142.0,54.97,NaN,15.50,61.0,4.11,3.34,M,NaN,1,1.0,VR,ATH
METROPOLIS.NS,1391.0,1.57,123.0,1365.98,NaN,49.80,44.0,17.40,15.20,L,NaN,0,0.0,VR,ATH
JMFINANCIL.NS,77.0,12.94,108.0,69.17,NaN,12.30,65.0,9.42,7.55,L,NaN,0,0.0,VR,2T3Y


In [14]:
# top stocks from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
CLEAN.NS,1286.0,-11.00,94.0,1365.11,NaN,46.3,28.0,44.50,33.2,L,NaN,0,1.0,VR,2T3Y
FINEORG.NS,4476.0,-13.01,56.0,4697.43,NaN,24.6,32.0,65.40,49.4,M,NaN,1,0.0,V40N,ATH
BANDHANBNK.NS,217.0,-7.62,176.0,244.52,NaN,17.2,39.0,6.67,11.9,M,NaN,1,0.0,VR,2T3Y
BAYERCROP.NS,4394.0,-0.64,38.0,4346.57,NaN,401.0,39.0,20.10,15.8,M,NaN,0,1.0,V40N,ATH
SRF.NS,2170.0,-7.73,23.0,2222.00,NaN,33.6,39.0,22.40,22.8,M,NaN,1,0.0,VR,ATH
TEAMLEASE.NS,2315.0,-5.16,97.0,2279.93,NaN,35.3,39.0,14.80,15.1,M,NaN,1,1.0,V40N,2T3Y
RAJESHEXPO.NS,512.0,-22.70,63.0,555.25,NaN,10.6,40.0,10.30,10.5,M,NaN,0,1.0,V40N,ATH
DMART.NS,3714.0,-0.72,37.0,3659.46,160.1,101.0,47.0,20.10,16.0,M,37.0,1,1.0,AR,ATH
INFY.NS,1340.0,-4.73,34.0,1324.07,29.0,22.5,48.0,40.70,31.8,H,22.0,1,1.0,V40,ATH


In [15]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PIDILITIND.NS,2616.0,4.09,6.0,2504.06,94.6,104.0,50.0,24.6,18.7,H,-10.0,0,1.0,V40,ATH
EICHERMOT.NS,3355.0,0.24,10.0,3260.20,44.7,31.5,48.0,27.4,21.1,H,30.0,0,1.0,V40N,ATH
5PAISA.NS,472.0,45.29,13.0,347.06,NaN,28.5,76.0,12.4,10.6,M,NaN,1,1.0,V40N,ATH
BAJAJFINSV.NS,1585.0,6.87,14.0,1454.93,39.2,35.8,49.0,12.7,14.8,H,9.0,1,1.0,V40,ATH
ICICIGI.NS,1374.0,17.21,16.0,1177.58,46.2,38.1,60.0,21.2,17.6,H,18.0,1,1.0,V40,ATH
TCS.NS,3355.0,3.56,16.0,3231.00,28.1,28.1,51.0,59.1,46.9,H,0.0,1,1.0,V40,ATH
BATAINDIA.NS,1716.0,7.71,17.0,1523.62,126.0,68.4,66.0,19.6,19.8,H,46.0,0,0.0,V40,ATH


In [16]:
df_common_features.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_common_features[~df_common_features.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]

In [29]:
# latest quarter prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(my_prospects)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,


In [18]:
# star + v40/v40n prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
DIXON.NS,4064.0,3653.0,11.24,5750.0,42.0,4050.0,25.8,22.4,V40N,NaN,5750.0,1,1.0,M,41.0
MOTILALOFS.NS,823.0,664.0,23.91,1091.0,71.0,818.0,13.0,15.6,V40N,NaN,1091.0,1,1.0,L,33.0
BERGEPAINT.NS,680.0,609.0,11.60,858.0,55.0,674.0,23.8,20.4,V40,NaN,858.0,0,1.0,M,26.0
ASIANPAINT.NS,3391.0,3035.0,11.72,3544.0,51.0,3369.0,35.2,27.7,V40,BTT,4250.0,1,1.0,H,25.0
BAJFINANCE.NS,7382.0,6551.0,12.68,7860.0,50.0,7284.0,11.8,23.5,V40,BTT,9150.0,1,1.0,H,24.0
ANGELONE.NS,1554.0,1353.0,14.89,1878.0,45.0,1591.0,44.0,47.1,V40N,NaN,1878.0,1,1.0,M,21.0
ASTRAZEN.NS,3867.0,3370.0,14.76,4631.0,67.0,3807.0,30.9,23.0,V40N,NaN,4631.0,0,1.0,M,20.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
AXISBANK.NS,950.0,898.0,5.79,985.0,44.0,960.0,6.16,14.5,V40,NaN,985.0,1,1.0,M,4.0
MCDOWELL-N.NS,1012.0,845.0,19.75,1039.0,66.0,997.0,19.90,17.5,V40N,NaN,1039.0,1,1.0,L,3.0
COLPAL.NS,2049.0,1569.0,30.61,2049.0,87.0,2002.0,180.00,158.0,V40,BTT,2100.0,1,1.0,H,2.0
AKZOINDIA.NS,2719.0,2277.0,19.43,2743.0,67.0,2743.0,34.70,26.1,V40,BTT,2743.0,0,1.0,M,1.0
BAJAJ-AUTO.NS,4892.0,3848.0,27.14,4918.0,64.0,4851.0,26.20,20.2,V40,NaN,4918.0,1,1.0,L,1.0
EQUITASBNK.NS,99.0,66.0,49.55,99.0,76.0,98.0,9.24,13.5,V40N,NaN,99.0,1,1.0,M,0.0
ICICIBANK.NS,996.0,904.0,10.20,997.0,70.0,990.0,6.82,18.7,V40,NaN,997.0,1,1.0,H,0.0
MARICO.NS,574.0,509.0,12.84,574.0,69.0,554.0,43.00,35.8,V40,NaN,574.0,1,1.0,M,0.0
POLYCAB.NS,4746.0,3063.0,54.94,4746.0,83.0,4701.0,27.10,20.0,V40N,NaN,4746.0,1,1.0,M,0.0


In [20]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 6


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
SBICARD.NS,857.0,807.0,6.22,1133.0,49.0,858.0,13.00,25.70,NaN,NaN,1133.0,0,1.0,M,32.0
UPL.NS,625.0,714.0,-12.44,832.0,31.0,627.0,15.80,13.50,NaN,NaN,832.0,0,1.0,L,33.0
GODREJIND.NS,475.0,447.0,6.30,644.0,42.0,476.0,9.75,11.20,NaN,NaN,644.0,0,1.0,L,36.0
ADANIENT.NS,2463.0,2697.0,-8.67,4161.0,59.0,2428.0,10.10,9.63,NaN,NaN,4161.0,0,1.0,M,69.0
ADANIGREEN.NS,1093.0,1325.0,-17.54,2970.0,69.0,1119.0,7.81,22.80,NaN,NaN,2970.0,0,1.0,L,172.0
ADANITRANS.NS,815.0,1670.0,-51.21,4105.0,57.0,805.0,10.50,11.60,NaN,NaN,4105.0,0,1.0,L,404.0


In [21]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
#print(tmp_df.index)
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
EQUITASBNK.NS,99.0,66.0,49.55,99.0,76.0,98.0,9.24,13.5,V40N,NaN,99.0,1,1.0,M,0.0
ICICIBANK.NS,996.0,904.0,10.20,997.0,70.0,990.0,6.82,18.7,V40,NaN,997.0,1,1.0,H,0.0
MARICO.NS,574.0,509.0,12.84,574.0,69.0,554.0,43.00,35.8,V40,NaN,574.0,1,1.0,M,0.0
POLYCAB.NS,4746.0,3063.0,54.94,4746.0,83.0,4701.0,27.10,20.0,V40N,NaN,4746.0,1,1.0,M,0.0
AKZOINDIA.NS,2719.0,2277.0,19.43,2743.0,67.0,2743.0,34.70,26.1,V40,BTT,2743.0,0,1.0,M,1.0
BAJAJ-AUTO.NS,4892.0,3848.0,27.14,4918.0,64.0,4851.0,26.20,20.2,V40,NaN,4918.0,1,1.0,L,1.0
COLPAL.NS,2049.0,1569.0,30.61,2049.0,87.0,2002.0,180.00,158.0,V40,BTT,2100.0,1,1.0,H,2.0
MCDOWELL-N.NS,1012.0,845.0,19.75,1039.0,66.0,997.0,19.90,17.5,V40N,NaN,1039.0,1,1.0,L,3.0
AXISBANK.NS,950.0,898.0,5.79,985.0,44.0,960.0,6.16,14.5,V40,NaN,985.0,1,1.0,M,4.0


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 2


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
NESTLEIND.NS,22481.0,20385.0,10.28,23138.0,45.0,22325.0,138.0,108.0,V40,NaN,23138.0,0,0.0,H,3.0
PGHH.NS,15971.0,13997.0,14.10,15986.0,68.0,15781.0,110.0,79.5,V40,NaN,15986.0,0,0.0,L,0.0


In [23]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 0


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,


In [24]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,953.0,1125.0,-15.30,3219.0,26.0,967.0,1000.00,1000.00,V40N,NaN,3219.0,1,0.0,M,238.0
TATAELXSI.NS,7223.0,6801.0,6.21,10602.0,36.0,7231.0,60.00,39.30,V40N,NaN,10602.0,0,0.0,M,47.0
DIXON.NS,4064.0,3653.0,11.24,5750.0,42.0,4050.0,25.80,22.40,V40N,NaN,5750.0,1,1.0,M,41.0
SYMPHONY.NS,868.0,941.0,-7.77,1468.0,42.0,875.0,14.80,13.50,V40N,NaN,1468.0,1,0.0,M,69.0
WHIRLPOOL.NS,1436.0,1427.0,0.60,2667.0,48.0,1417.0,8.87,6.37,V40,NaN,2667.0,0,0.0,M,86.0
SFL.NS,1190.0,1192.0,-0.16,1972.0,54.0,1175.0,13.60,12.50,V40N,NaN,1972.0,0,0.0,M,66.0
LALPATHLAB.NS,2355.0,2157.0,9.16,4186.0,55.0,2482.0,18.50,15.00,V40N,NaN,4186.0,1,0.0,M,78.0
TTKPRESTIG.NS,777.0,779.0,-0.28,1156.0,60.0,771.0,18.20,13.80,V40N,NaN,1156.0,0,0.0,M,49.0
PFIZER.NS,3978.0,4011.0,-0.82,6009.0,71.0,3895.0,27.20,19.80,V40,NaN,6009.0,0,0.0,L,51.0


In [25]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,953.0,1125.0,-15.30,3219.0,26.0,967.0,1000.00,1000.00,V40N,NaN,3219.0,1,0.0,M,238.0
TATAELXSI.NS,7223.0,6801.0,6.21,10602.0,36.0,7231.0,60.00,39.30,V40N,NaN,10602.0,0,0.0,M,47.0
DIXON.NS,4064.0,3653.0,11.24,5750.0,42.0,4050.0,25.80,22.40,V40N,NaN,5750.0,1,1.0,M,41.0
SYMPHONY.NS,868.0,941.0,-7.77,1468.0,42.0,875.0,14.80,13.50,V40N,NaN,1468.0,1,0.0,M,69.0
WHIRLPOOL.NS,1436.0,1427.0,0.60,2667.0,48.0,1417.0,8.87,6.37,V40,NaN,2667.0,0,0.0,M,86.0
SFL.NS,1190.0,1192.0,-0.16,1972.0,54.0,1175.0,13.60,12.50,V40N,NaN,1972.0,0,0.0,M,66.0
LALPATHLAB.NS,2355.0,2157.0,9.16,4186.0,55.0,2482.0,18.50,15.00,V40N,NaN,4186.0,1,0.0,M,78.0
TTKPRESTIG.NS,777.0,779.0,-0.28,1156.0,60.0,771.0,18.20,13.80,V40N,NaN,1156.0,0,0.0,M,49.0
PFIZER.NS,3978.0,4011.0,-0.82,6009.0,71.0,3895.0,27.20,19.80,V40,NaN,6009.0,0,0.0,L,51.0


In [26]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_common_features[df_common_features.index == stock_id]

,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1529.0,1595.0,-4.14,1917.0,68.0,1530.0,1000.0,1000.0,NaN,NaN,1917.0,0,0.0,L,25.0


In [27]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [28]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-06-30 00:00:00+05:30,678.0,605.0,12.08,BERGEPAINT.NS,858.0,67.0
2023-07-07 00:00:00+05:30,673.0,605.0,11.21,BERGEPAINT.NS,858.0,57.0
2023-07-14 00:00:00+05:30,668.0,606.0,10.25,BERGEPAINT.NS,858.0,52.0
2023-07-21 00:00:00+05:30,691.0,607.0,13.86,BERGEPAINT.NS,858.0,67.0
2023-07-28 00:00:00+05:30,680.0,609.0,11.60,BERGEPAINT.NS,858.0,55.0
